In [0]:
from pyspark.sql.functions import (
    col, coalesce, lit, when, current_timestamp, length, levenshtein, greatest, lower, substring, row_number
)
from pyspark.sql import Window

In [0]:
acra_df = spark.read.table("silver.clean.acra").alias("a")
recordowl_df = spark.read.table("silver.clean.recordowl").alias("r")
scraped_df = spark.read.table("silver.clean.scrapped_wesbites").alias("s")
companies_sg_df = spark.read.table("silver.clean.companies_sg").alias("sg")
stocks_df = spark.read.table("silver.clean.stocks").alias("st")

In [0]:
unified = (
    acra_df
    .join(companies_sg_df, col("a.uen") == col("sg.uen_match"), "left")
    .join(recordowl_df, col("a.uen") == col("r.uen_match"), "left")
    .join(scraped_df, col("a.uen") == col("s.uen_match"), "left")
    .drop("uen_match")
)
display(unified)

# Fuzzy join with Stocks

In [0]:
stock_match = stocks_df.alias("st").join(
    unified.select("uen", "company_name").alias("u"),
    lower(substring(col("st.stock_company_name"), 1, 4)) == lower(substring(col("u.company_name"), 1, 4)),
    "inner"
)

display(stock_match)

In [0]:
stock_match = stock_match.withColumn(
    "similarity_score",
    1 - (levenshtein(col("st.stock_company_name"), col("u.company_name")) /
        greatest(length(col("st.stock_company_name")), length(col("u.company_name"))))
).filter(col("similarity_score") > 0.85)

display(stock_match)

In [0]:
window = Window.partitionBy("stock_symbol").orderBy(col("similarity_score").desc())
best_stock = stock_match.withColumn("rank", row_number().over(window)).filter(col("rank") == 1)
display(best_stock)

In [0]:
unified = unified.join(
    best_stock.select("uen", "stock_symbol", "market_cap", "revenue", "stock_price", "percent_change"),
    "uen",
    "left"
)
display(unified)

In [0]:
final_unified = unified.select(
    col("a.uen"),
    coalesce(col("a.company_name"), col("r.owl_company_name"), col("sg.sg_company_name"), col("s.scraped_company_name")).alias("company_name"),

    coalesce(
        col("r.recordowl_website"),
        col("s.recordowl_website")
    ).alias("website"),

    coalesce(col("a.industry_description"), col("r.owl_industry")).alias("industry"),
    coalesce(col("a.industry_code"), col("r.owl_ssic_code")).alias("industry_code"),
    coalesce(col("a.secondary_ssic_description"), col("r.owl_secondary_industry")).alias("secondary_industry"),
    coalesce(col("a.entity_status_description"), col("sg.sg_entity_status")).alias("entity_status"),
    coalesce(col("a.entity_type_description"), col("sg.sg_company_type")).alias("company_type"),

    col("a.address"),
    col("a.founding_year"),
    col("a.no_of_officers"),

    coalesce(col("r.linkedin_url"), col("s.scraped_linkedin")).alias("linkedin"),
    coalesce(col("r.facebook_url"), col("s.scraped_facebook")).alias("facebook"),
    coalesce(col("r.instagram_url"), col("s.scraped_instagram")).alias("instagram"),

    col("s.scraped_email").alias("contact_email"),
    coalesce(col("r.phone_number"), col("s.scraped_phone")).alias("contact_phone"),

    col("r.company_description"),
    col("s.scraped_keywords").alias("keywords"),

    col("stock_symbol"),
    col("market_cap"),#.cast("double"),
    col("revenue"),#.cast("double"),
    col("stock_price"),#.cast("double"),
    col("percent_change"),#.cast("double"),

    when(col("stock_symbol").isNotNull() & (col("percent_change").isNotNull()), False).otherwise(True).alias("is_it_delisted"),
    lit("Singapore").alias("hq_country"),
    lit(1).alias("no_of_locations_in_singapore"),

    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
)

In [0]:
display(final_unified)

In [0]:
final_unified=final_unified.dropDuplicates(['uen'])
display(final_unified)

## Merge Command Unified Data

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import current_timestamp

gold_path = "abfss://silver@singaporecomadls.dfs.core.windows.net/unified_companies"

# Add or refresh updated_at timestamp before merge
final_unified_upsert = final_unified.withColumn("updated_at", current_timestamp())

# Load the existing Delta table
delta_target = DeltaTable.forPath(spark, gold_path)

# Perform the MERGE operation
(
    delta_target.alias("t")
    .merge(final_unified_upsert.alias("s"), "t.uen = s.uen")
    .whenMatchedUpdate(set={
        "company_name": "s.company_name",
        "website": "s.website",
        "industry": "s.industry",
        "industry_code": "s.industry_code",
        "secondary_industry": "s.secondary_industry",
        "entity_status": "s.entity_status",
        "company_type": "s.company_type",
        "address": "s.address",
        "founding_year": "s.founding_year",
        "no_of_officers": "s.no_of_officers",
        "linkedin": "s.linkedin",
        "facebook": "s.facebook",
        "instagram": "s.instagram",
        "contact_email": "s.contact_email",
        "contact_phone": "s.contact_phone",
        "company_description": "s.company_description",
        "keywords": "s.keywords",
        "stock_symbol": "s.stock_symbol",
        "market_cap": "s.market_cap",
        "revenue": "s.revenue",
        "stock_price": "s.stock_price",
        "percent_change": "s.percent_change",
        "is_it_delisted": "s.is_it_delisted",
        "hq_country": "s.hq_country",
        "no_of_locations_in_singapore": "s.no_of_locations_in_singapore",
        "updated_at": "current_timestamp()"
    })
    .whenNotMatchedInsert(values={
        "uen": "s.uen",
        "company_name": "s.company_name",
        "website": "s.website",
        "industry": "s.industry",
        "industry_code": "s.industry_code",
        "secondary_industry": "s.secondary_industry",
        "entity_status": "s.entity_status",
        "company_type": "s.company_type",
        "address": "s.address",
        "founding_year": "s.founding_year",
        "no_of_officers": "s.no_of_officers",
        "linkedin": "s.linkedin",
        "facebook": "s.facebook",
        "instagram": "s.instagram",
        "contact_email": "s.contact_email",
        "contact_phone": "s.contact_phone",
        "company_description": "s.company_description",
        "keywords": "s.keywords",
        "stock_symbol": "s.stock_symbol",
        "market_cap": "s.market_cap",
        "revenue": "s.revenue",
        "stock_price": "s.stock_price",
        "percent_change": "s.percent_change",
        "is_it_delisted": "s.is_it_delisted",
        "hq_country": "s.hq_country",
        "no_of_locations_in_singapore": "s.no_of_locations_in_singapore",
        "created_at": "current_timestamp()",
        "updated_at": "current_timestamp()"
    })
    .execute()
)


In [0]:
%sql
SELECT * FROM silver.unified.unified_companies

In [0]:
final_unified = spark.read.table("silver.unified.unified_companies")

# Find Coverage

In [0]:
final_unified = spark.table("silver.unified.unified_companies")

In [0]:
final_unified = final_unified.withColumn(
    "data_completeness_score",
    (
        when(col("website").isNotNull(), 1).otherwise(0) +
        when(col("linkedin").isNotNull(), 1).otherwise(0) +
        when(col("facebook").isNotNull(), 1).otherwise(0) +
        when(col("instagram").isNotNull(), 1).otherwise(0) +
        when(col("contact_email").isNotNull(), 1).otherwise(0) +
        when(col("contact_phone").isNotNull(), 1).otherwise(0) +
        when(col("revenue").isNotNull(), 1).otherwise(0) +
        when(col("keywords").isNotNull(), 1).otherwise(0)
    ) / 8.0 * 100
)

print(f"✓ Final unified records: {final_unified.count():,}")

In [0]:
from pyspark.sql.functions import count, countDistinct, avg# Coverage statistics

coverage = final_unified.select(
    count("*").alias("total_companies"),
    countDistinct("uen").alias("unique_uens"),
    (count("website") / count("*") * 100).alias("website_%"),
    (count("linkedin") / count("*") * 100).alias("linkedin_%"),
    (count("facebook") / count("*") * 100).alias("facebook_%"),
    (count("instagram") / count("*") * 100).alias("instagram_%"),
    (count("contact_email") / count("*") * 100).alias("email_%"),
    (count("contact_phone") / count("*") * 100).alias("phone_%"),
    (count("revenue") / count("*") * 100).alias("revenue_%"),
    (count("company_type") / count("*") * 100).alias("company_type_%"),
    avg("data_completeness_score").alias("avg_completeness")
)

In [0]:
display(coverage)

In [0]:
%run "/Repos/24huda.zaidi@fostiima.org/DE-Singapore-Companies-DB/Pyspark_Notebooks/Cleaning/data_quality_utils"

In [0]:

df = spark.read.format("delta").load(
    "abfss://silver@singaporecomadls.dfs.core.windows.net/unified_companies/"
)

#Step 1: Add completeness score
df = calculate_data_completeness_score(df)

#Step 2: Run data quality checks
df = run_all_quality_checks(df)

#Step 3: Export reports
export_matching_report(
    df, "abfss://silver@singaporecomadls.dfs.core.windows.net/export_matching_report/"
)
export_statistics(
    df, "abfss://silver@singaporecomadls.dfs.core.windows.net/export_statistics/"
)

In [0]:
export_matching_report = spark.read.format('csv').option('header', 'true').load(
    "abfss://silver@singaporecomadls.dfs.core.windows.net/export_matching_report/matching_quality_report"
)
display(export_matching_report)

In [0]:
export_statistics = spark.read.format('csv').option("header", "true").load(
    "abfss://silver@singaporecomadls.dfs.core.windows.net/export_statistics/summary_statistics"
)
display(export_statistics)